# SPT analysis - Interactive report
###### AB @ BCB ITQB 


### Instructions:
1. To run a cell select it and press **SHIFT+ENTER**;
2. All cells require the **previous** one to be run first;
3. If you change the loaded file through the dropdown menu, **RUN** the interactive graph cell (the next one) again to update the data 
4. If you make a mistake in the order or just want to restart the report **restart the kernel** and start from the beginning
5. Reading a .npy file takes a few seconds depending on its size. The loading is cached, meaning that loading a previously loaded file is instant.
---

### Import required functions

In [ ]:
%matplotlib widget
from ReportJupyter import *

### Choose folder that contains all npy's

In [ ]:
# Choose folder
root = Tk()
rootdir = filedialog.askdirectory()
root.destroy()

load_widget = interactive(FileLoader, 
                          folder=Dropdown(options=os.listdir(rootdir), description='Experimental conditions:', disabled=False), 
                          root = fixed(rootdir))
display(load_widget)

out = Output()
button = Button(description="Click here to preload ALL data")
button.on_click(partial(loadall, rootdir, out))
display(button)
display(out)


### Interactive filtering
- Filter the tracks by assigning a maximum allowed angle;
- This angle refers to the deviation of the Z-ring ellipse to a perfect circle
- Higher angles correspond to slanted rings and higher errors

In [ ]:
TrackObjects = np.array(load_widget.result)

#diameter = np.array([tr.ellipse['major']*1000 for tr in TrackObjects])
#angle = np.rad2deg(np.arccos([i.ellipse['minor']/i.ellipse['major'] for i in TrackObjects]))

angle_slider = IntRangeSlider(value=[0, 60], min=0, max=90, step=1, description='Angle:')
major_slider = IntRangeSlider(value=[0, 800], min=0, max=2000, step=10, description='Diameter:')

angle_wid = interactive(Update_Graphs, angle_threshold=angle_slider, major_threshold=major_slider,
            all_tracks=fixed(TrackObjects))

angle_wid


### Interactive comparison
- After defining appropriate filtering in the above cells select tracks for comparison
- By default, adding a ftsW conditions WILL look for the corresponding divIB

In [ ]:
selections = SelectMultiple(
    options=os.listdir(rootdir),
    description='Available Tracks',
    rows=len(os.listdir(rootdir)),
    width=200,
    disabled=False
)



angle = BoundedIntText(
    value=90,
    min=0,
    max=91,
    step=1,
    description='Angle Threshold:',
    disabled=False
)

comparison_wid = interactive(ViolinComparison, conditions=selections, root=fixed(rootdir), anglethresh=angle)
comparison_wid